In [38]:
from tba_session import session, ROOT_URL, TEAM_PAGES
from tba_types import Award, Event, Team
from collections import defaultdict
from tqdm.notebook import trange, tqdm
from itertools import pairwise

In [39]:
ALL_TEAMS: dict[str, Team] = {}

for page in trange(TEAM_PAGES, desc="Getting Teams", total=TEAM_PAGES, leave=False):
    team: Team
    for team in session.get(ROOT_URL + f"/teams/{page}").json():
        ALL_TEAMS[team["key"]] = team

Getting Teams:   0%|          | 0/22 [00:00<?, ?it/s]

In [43]:
einstein_years: dict[str, list[int]] =  defaultdict(list)
cmp_years: dict[str, list[int]] = defaultdict(list)
team_seasons: dict[str, set[int]] = defaultdict(set)

for year in trange(2001, 2026, desc="Fetching years"):
    events: list[Event] = session.get(f"{ROOT_URL}/events/{year}").json()
    for event in tqdm(events, leave=False, desc=f"Processing {year} events"):
        if event["event_type"] == 3:
            awards: list[Award] = session.get(f"{ROOT_URL}/event/{event['key']}/awards").json()
            for award in awards:
                if award["award_type"] == 1:
                    for recipient in award["recipient_list"]:
                        team_key = recipient["team_key"]
                        einstein_years[team_key].append(year)
            teams: list[str] = session.get(f"{ROOT_URL}/event/{event['key']}/teams/keys").json()
            for team_key in teams:
                cmp_years[team_key].append(year)
        elif event["event_type"] <= 6 and event["year"] != 2020 and event["year"] != 2021:
            teams: list[str] = session.get(f"{ROOT_URL}/event/{event['key']}/teams/keys").json()
            for team_key in teams:
                team_seasons[team_key].add(year)

Fetching years:   0%|          | 0/25 [00:00<?, ?it/s]

Processing 2001 events:   0%|          | 0/18 [00:00<?, ?it/s]

Processing 2002 events:   0%|          | 0/22 [00:00<?, ?it/s]

Processing 2003 events:   0%|          | 0/28 [00:00<?, ?it/s]

Processing 2004 events:   0%|          | 0/32 [00:00<?, ?it/s]

...

Processing 2021 events:   0%|          | 0/259 [00:00<?, ?it/s]

Processing 2022 events:   0%|          | 0/288 [00:00<?, ?it/s]

Processing 2023 events:   0%|          | 0/309 [00:00<?, ?it/s]

Processing 2024 events:   0%|          | 0/323 [00:00<?, ?it/s]

Processing 2025 events:   0%|          | 0/222 [00:00<?, ?it/s]

In [ ]:
print ("| # | Team Number | Team Name | First Year | Second Year | Gap |")
print( "| --- | --- | --- | --- | --- | --- |")

for idx, vals in enumerate(sorted(
    (
        (team_key, comp_years[0], comp_years[1], comp_years[1] - comp_years[0])
        for team_key, comp_years in einstein_years.items()
        if len(comp_years) >= 2
        for comp_years in pairwise(sorted(comp_years))
    ),
    key=lambda x: x[3],
    reverse=True
)[:25], start=1):
    team_key, first, second, gap = vals
    team = ALL_TEAMS[team_key]
    print(
        f"| {idx} | {team['team_number']} | {team['nickname']} | "
        f"{first} | {second} | {gap} year gap |"
    )

| # | Team Number | Team Name | First Year | Second Year | Gap |
| --- | --- | --- | --- | --- | --- |
| 1 | 59 | RamTech | 2001 | 2023 | 22 year gap |
| 2 | 190 | Gompei and the HERD | 2007 | 2024 | 17 year gap |
| 3 | 125 | NUTRONs | 2001 | 2017 | 16 year gap |
| 4 | 294 | Beach Cities Robotics | 2010 | 2024 | 14 year gap |
| 5 | 494 | Martians | 2004 | 2018 | 14 year gap |
| 6 | 302 | The Dragons | 2003 | 2017 | 14 year gap |
| 7 | 868 | TechHOUNDS | 2004 | 2018 | 14 year gap |
| 8 | 144 | Operation Orange | 2002 | 2015 | 13 year gap |
| 9 | 180 | S.P.A.M. | 2012 | 2025 | 13 year gap |
| 10 | 16 | Bomb Squad | 2012 | 2025 | 13 year gap |
| 11 | 179 | Children of the Swamp | 2007 | 2019 | 12 year gap |
| 12 | 118 | Robonauts | 2002 | 2012 | 10 year gap |
| 13 | 180 | S.P.A.M. | 2002 | 2012 | 10 year gap |
| 14 | 1218 | SCH Robotics | 2009 | 2019 | 10 year gap |
| 15 | 195 | CyberKnights | 2006 | 2016 | 10 year gap |
| 16 | 971 | Spartan Robotics | 2009 | 2019 | 10 year gap |
| 17 | 1

In [ ]:
print ("| # | Team Number | Team Name | Number of Championship Appearances |")
print( "| --- | --- | --- | --- |")

for idx, (team_key, years) in enumerate(
    sorted(
        ((team_key, len(years)) for team_key, years in cmp_years.items() if len(years) > 0 and team_key not in einstein_years),
        key=lambda x: x[1],
        reverse=True
    )[:15], start=1
):
    team = ALL_TEAMS[team_key]
    print(
        f"| {idx} | {team['team_number']} | {team['nickname']} | "
        f"{years} appearances |"
    )

| # | Team Number | Team Name | Number of Championship Appearances |
| --- | --- | --- | --- |
| 1 | 45 | TechnoKats Robotics Team | 20 appearances |
| 2 | 399 | Eagle Robotics | 19 appearances |
| 3 | 103 | Cybersonics | 19 appearances |
| 4 | 176 | Aces High | 18 appearances |
| 5 | 11 | MORT | 18 appearances |
| 6 | 88 | TJ² | 18 appearances |
| 7 | 126 | Gael Force | 17 appearances |
| 8 | 191 | X-CATS | 17 appearances |
| 9 | 1538 | The Holy Cows | 17 appearances |
| 10 | 1629 | Garrett Coalition (GaCo) | 17 appearances |
| 11 | 573 | Mech Warriors | 16 appearances |
| 12 | 842 | Falcon Robotics | 16 appearances |
| 13 | 836 | The RoboBees | 16 appearances |
| 14 | 1305 | Ice Cubed | 16 appearances |
| 15 | 1816 | "The Green Machine" | 16 appearances |


In [44]:
print("| # | Team Number | Team Name | Einstein/Worlds |")
print("| --- | --- | --- | --- |")

for idx, (team_key, ratio) in enumerate(
    sorted(
        ((team_key, len(einstein_years[team_key]) / len(cmp_years[team_key]))
         for team_key in cmp_years if team_key in einstein_years and len(cmp_years[team_key]) > 2),
        key=lambda x: x[1],
        reverse=True
    )[:15], start=1
):
    team = ALL_TEAMS[team_key]
    print(
        f"| {idx} | {team['team_number']} | [{team['nickname']}](https://thebluealliance.com/team/{team["team_number"]}) | "
        f"{ratio*100:.2f}% |"
    )

| # | Team Number | Team Name | Einstein/Worlds |
| --- | --- | --- | --- |
| 1 | 1678 | [Citrus Circuits](https://thebluealliance.com/team/1678) | 76.92% |
| 2 | 3707 | [TechnoDogs](https://thebluealliance.com/team/3707) | 66.67% |
| 3 | 4414 | [HighTide](https://thebluealliance.com/team/4414) | 60.00% |
| 4 | 64 | [Gila Monsters](https://thebluealliance.com/team/64) | 50.00% |
| 5 | 5026 | [Iron Panthers](https://thebluealliance.com/team/5026) | 50.00% |
| 6 | 217 | [ThunderChickens](https://thebluealliance.com/team/217) | 47.37% |
| 7 | 2056 | [OP Robotics](https://thebluealliance.com/team/2056) | 47.06% |
| 8 | 254 | [The Cheesy Poofs](https://thebluealliance.com/team/254) | 45.45% |
| 9 | 3476 | [Code Orange](https://thebluealliance.com/team/3476) | 45.45% |
| 10 | 144 | [Operation Orange](https://thebluealliance.com/team/144) | 42.86% |
| 11 | 6328 | [Mechanical Advantage](https://thebluealliance.com/team/6328) | 42.86% |
| 12 | 973 | [Greybots](https://thebluealliance.com/team/9

In [48]:
print("| # | Team Number | Team Name | Einstein/Total |")
print("| --- | --- | --- | --- |")

for idx, (team_key, ratio) in enumerate(
    sorted(
        ((team_key, len(einstein_years[team_key]) / len(team_seasons[team_key]))
         for team_key in cmp_years if team_key in einstein_years and len(team_seasons[team_key]) > 2),
        key=lambda x: x[1],
        reverse=True
    )[:15], start=1
):
    team = ALL_TEAMS[team_key]
    print(
        f"| {idx} | {team['team_number']} | [{team['nickname']}](https://thebluealliance.com/team/{team["team_number"]}) | "
        f"{ratio*100:.2f}% |"
    )

| # | Team Number | Team Name | Einstein/Total |
| --- | --- | --- | --- |
| 1 | 4414 | [HighTide](https://thebluealliance.com/team/4414) | 60.00% |
| 2 | 1678 | [Citrus Circuits](https://thebluealliance.com/team/1678) | 52.63% |
| 3 | 2056 | [OP Robotics](https://thebluealliance.com/team/2056) | 47.06% |
| 4 | 254 | [The Cheesy Poofs](https://thebluealliance.com/team/254) | 43.48% |
| 5 | 6328 | [Mechanical Advantage](https://thebluealliance.com/team/6328) | 42.86% |
| 6 | 64 | [Gila Monsters](https://thebluealliance.com/team/64) | 40.00% |
| 7 | 217 | [ThunderChickens](https://thebluealliance.com/team/217) | 39.13% |
| 8 | 3476 | [Code Orange](https://thebluealliance.com/team/3476) | 38.46% |
| 9 | 6986 | [PPT Bots](https://thebluealliance.com/team/6986) | 33.33% |
| 10 | 9072 | [TigerBots](https://thebluealliance.com/team/9072) | 33.33% |
| 11 | 9312 | [NERD Spark](https://thebluealliance.com/team/9312) | 33.33% |
| 12 | 987 | [HIGHROLLERS](https://thebluealliance.com/team/987) | 31

In [ ]:
for team_key, last in sorted(
    (
        (team_key, comp_years[-1])
        for team_key, comp_years in einstein_years.items()
        if len(comp_years) >= 1
    ),
    key=lambda x: x[1],
)[:20]:
    team = ALL_TEAMS[team_key]
    print(f"{team['team_number']:>4} {team['nickname']:<30} {last:>4} (last competed)")

 349 Team 349                       2001 (last competed)
  75 RoboRaiders                    2001 (last competed)
  85 B.O.B. (Built on Brains)       2001 (last competed)
 115 MVRT                           2001 (last competed)
  53 Area 53                        2001 (last competed)
 122 NASA Knights                   2001 (last competed)
 279 Tech Fusion                    2001 (last competed)
 365 Miracle Workerz                2001 (last competed)
 308 Monsters                       2002 (last competed)
 311 Red Jammers                    2002 (last competed)
  66 Grizzly Robotics               2002 (last competed)
  65 The Huskie Brigade             2003 (last competed)
 343 Metal-In-Motion                2003 (last competed)
 236 Techno-Ticks                   2003 (last competed)
 341 Miss Daisy                     2003 (last competed)
 292 PantherTech                    2003 (last competed)
 378 The Circuit Stompers           2003 (last competed)
 340 G.R.R. (Greater Rochester 